In [ ]:
from datetime import datetime
from os import environ

environ['DATA_DOWNLOAD'] = 'true'

if 'RUN_DIR' not in environ:
	environ['RUN_DIR'] = f"./runs/{datetime.now().astimezone().strftime('%Y-%m-%d %H-%M-%S-%f %z')}/"

print(f"RUN_DIR={environ['RUN_DIR']}")

In [ ]:
import asyncio

from node import build as build_node
from torch.optim.lr_scheduler import StepLR
from pathlib import Path


async def node_amain(name: str):
	node = await build_node(
		name=name,
		data_distribution='stratified',
		communicator='local',
		data_dir_path=Path(f"{environ['RUN_DIR']}/data/{name}/"),
	)
	
	lr_stepper = StepLR(node._torch_context.optimizer, step_size=1, gamma=0.9)
	
	while True:
		await node.step_round()
		lr_stepper.step()

In [ ]:
async with asyncio.TaskGroup() as tg:
	for i in range(10):
		name = f"node-{i}"
		tg.create_task(node_amain(name))